In [ ]:
import sys
sys.path.append('/home/meron/Documents/work/tenacademy/week1/eda')
import pandas as pd
from scripts.process_data import get_df
import numpy as np
import seaborn as sns
pd.set_option('display.float_format', lambda x: '%.0f' % x)
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [ ]:
df = get_df()

In [ ]:
"""Task 4. 1 - Aggregate, per customer, the following information (treat missing & outliers by replacing by 
the mean or the mode of the corresponding variable):
"""
analytics_columns = [
    'MSISDN/Number',
    'Avg RTT DL (ms)',
    'Avg RTT UL (ms)',
    'TCP DL Retrans. Vol (Bytes)',
    'TCP UL Retrans. Vol (Bytes)',
    'Handset Type',

    #through put
    'Avg Bearer TP DL (kbps)',
    'Avg Bearer TP UL (kbps)',
]

num_columns = [
    'Avg RTT DL (ms)',
    'Avg RTT UL (ms)',
    'TCP DL Retrans. Vol (Bytes)',
    'TCP UL Retrans. Vol (Bytes)',
    #through put
    'Avg Bearer TP DL (kbps)',
    'Avg Bearer TP UL (kbps)',
]

df = df[analytics_columns]
# drop the NaNs of handset type column becuase it is a string field and doesn't have mean
df_cleaned_handset=df.dropna(subset=['Handset Type'])

df_cleaned_handset 

In [ ]:

lower_percentile = 1
upper_percentile = 99

lower_bounds = df_cleaned_handset[num_columns].quantile(lower_percentile / 100)
upper_bounds = df_cleaned_handset[num_columns].quantile(upper_percentile / 100)

for col in num_columns:
    outliers = (df_cleaned_handset[col] < lower_bounds[col]) | (df_cleaned_handset[col] > upper_bounds[col])
    if col  == 'Avg Bearer TP DL (kbps)':
    
     df_cleaned_handset[col] = df_cleaned_handset[col].mask(outliers, df_cleaned_handset[col].mean())

df_cleaned_handset



In [ ]:
# replace the NaN value by mean
df_clean_nan = df_cleaned_handset.copy()
df_clean_nan[num_columns] = df_clean_nan[num_columns].fillna(df_clean_nan[num_columns].mean())
df_clean_nan

In [ ]:
# Create sum columns for each UL and DL pair

df_clean_nan['Sum RTT'] = df_clean_nan['Avg RTT DL (ms)'] + df_clean_nan['Avg RTT UL (ms)']
df_clean_nan['Sum TCP Retrans. Vol (Bytes)'] = df_clean_nan['TCP DL Retrans. Vol (Bytes)'] + df_clean_nan['TCP UL Retrans. Vol (Bytes)']
df_clean_nan['Sum Bearer TP'] = df_clean_nan['Avg Bearer TP DL (kbps)'] + df_clean_nan['Avg Bearer TP UL (kbps)']

# Rename columns without DL and UL
df_clean_nan = df_clean_nan.rename(columns={
    'Avg RTT DL (ms)': 'Avg RTT DL',
    'Avg RTT UL (ms)': 'Avg RTT UL',
    'TCP DL Retrans. Vol (Bytes)': 'TCP Retrans. Vol DL',
    'TCP UL Retrans. Vol (Bytes)': 'TCP Retrans. Vol UL',
    'Avg Bearer TP DL (kbps)': 'Avg Bearer TP DL',
    'Avg Bearer TP UL (kbps)': 'Avg Bearer TP UL',
})

df_clean_nan


In [ ]:
important_columns = [
    'MSISDN/Number',
    'Handset Type',
    'Sum RTT',
    'Sum TCP Retrans. Vol (Bytes)',
    'Sum Bearer TP'
]

num_important_columns  = [
    'Sum RTT',
    'Sum TCP Retrans. Vol (Bytes)',
    'Sum Bearer TP'
]

df_important = df_clean_nan[important_columns]
df_important

In [ ]:
"""Aggregate, per customer, the following information (treat missing & outliers by replacing by the mean or the mode of the corresponding variable):
 Average TCP retransmission
 Average RTT
 Handset type
 Average throughput"""

# List of columns to include in the groupby and sum
num_important_columns = [
    'Sum RTT',
    'Sum TCP Retrans. Vol (Bytes)',
    'Sum Bearer TP'
]

# Group by 'MSISDN/Number' and sum the values for each group
grouped_df = df_important.groupby('MSISDN/Number')[num_important_columns].sum().reset_index()

# Display the grouped DataFrame
grouped_df.head(10)


In [ ]:
"""Top 1bottom and most frequent TCP values"""

top_tcp_values = df_important['Sum TCP Retrans. Vol (Bytes)'].nlargest(10)
bottom_tcp_values = df_important['Sum TCP Retrans. Vol (Bytes)'].nsmallest(10)
most_frequent_tcp_values = df_important['Sum TCP Retrans. Vol (Bytes)'].mode()

print("Top TCP Values:")
print(top_tcp_values)

print("\nBottom TCP Values:")
print(bottom_tcp_values)

print("\nMost Frequent TCP Values:")
print(most_frequent_tcp_values)



In [ ]:

"""Top 1bottom and most frequent RTT values"""
top_rtt_values = df_important['Sum RTT'].nlargest(10)
bottom_rtt_values = df_important['Sum RTT'].nsmallest(10)
most_frequent_rtt_values = df_important['Sum RTT'].mode()

print("Top RTT Values:")
print(top_rtt_values)

print("\nBottom RTT Values:")
print(bottom_rtt_values)

print("\nMost Frequent RTT Values:")
print(most_frequent_rtt_values)



In [ ]:
"""Top bottom and most frequent 10 Throughput values"""
top_throughput_values = df_important['Sum Bearer TP'].nlargest(10)
bottom_throughput_values = df_important['Sum Bearer TP'].nsmallest(10)
most_frequent_throughput_values = df_important['Sum Bearer TP'].mode()

print("Top Throughput Values:")
print(top_throughput_values)

print("\nBottom Throughput Values:")
print(bottom_throughput_values)

print("\nMost Frequent Throughput Values:")
print(most_frequent_throughput_values)



In [ ]:
# Distribution of the average throughput per handset type
average_throughput_per_handset = df_important.groupby('Handset Type')['Sum Bearer TP'].mean().reset_index()
print("Distribution of Average Throughput per Handset Type:")
print(average_throughput_per_handset)

# Distribution of the avg TCP retransmission view per handset type
avg_tcp_r_per_handset = df_important.groupby('Handset Type')['Sum TCP Retrans. Vol (Bytes)'].mean().reset_index()
print("Distribution of Average TCP Retransmission per Handset Type:")
avg_tcp_r_per_handset


In [ ]:

# Select relevant columns for clustering
experience_metrics = [
    'Sum RTT',
    'Sum TCP Retrans. Vol (Bytes)',
    'Sum Bearer TP',
]

# Drop rows with missing values in the selected columns
df_cluster = grouped_df[experience_metrics]

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_cluster)

# Apply k-means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
df_cluster['Cluster'] = kmeans.fit_predict(scaled_data)

# Analyze cluster characteristics
cluster_means = df_cluster.groupby('Cluster').mean()
print(cluster_means)


In [ ]:

# Select relevant columns for clustering
experience_metrics = [
    'Sum RTT',
    'Sum TCP Retrans. Vol (Bytes)',
    'Sum Bearer TP',
]

# Drop rows with missing values in the selected columns
df_cluster = grouped_df[experience_metrics].dropna()

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_cluster)

# Apply k-means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
df_cluster['Cluster'] = kmeans.fit_predict(scaled_data)

# Visualize the clusters using scatter plots
sns.set(style="whitegrid")

# Pairwise scatter plots for different pairs of experience metrics
sns.pairplot(df_cluster, hue="Cluster", palette="Set1", height=3, diag_kind="kde")
plt.show()


In [ ]:
# to visiualize by 3D
experience_metrics = [
    'Sum RTT',
    'Sum TCP Retrans. Vol (Bytes)',
    'Sum Bearer TP',
]

# Drop rows with missing values in the selected columns
df_cluster = grouped_df[experience_metrics].dropna()

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_cluster)

# Apply k-means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
df_cluster['Cluster'] = kmeans.fit_predict(scaled_data)

# Visualize the clusters using a 3D scatter plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(
    df_cluster['Sum RTT'],
    df_cluster['Sum TCP Retrans. Vol (Bytes)'],
    df_cluster['Sum Bearer TP'],
    c=df_cluster['Cluster'],
    cmap='viridis',
    s=50,
    alpha=0.6,
    edgecolors='w'
)

ax.set_xlabel('Sum RTT')
ax.set_ylabel('Sum TCP Retrans. Vol (Bytes)')
ax.set_zlabel('Sum Bearer TP')
ax.set_title('K-Means Clustering of User Experiences')

legend1 = ax.legend(*scatter.legend_elements(), title='Clusters')
ax.add_artist(legend1)

plt.show()
